# 0. 标定
- 标定板
- 相机： Intel RealSense D405 

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import time

context = rs.context()
devices = context.query_devices()
for dev in devices:
    print(dev.get_info(rs.camera_info.name))

Intel RealSense D405


In [8]:
# 创建管道并配置
pipeline = rs.pipeline()
config = rs.config()

config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# 启用红外流并配置Y16格式
# config.enable_stream(rs.stream.infrared, 1, 848, 480, rs.format.y16, 15)
 
# 启动管道
pipe_profile = pipeline.start(config)

In [ ]:
# 获取出厂内参矩阵
profile = pipe_profile.get_stream(rs.stream.color)
intrinsics = profile.as_video_stream_profile().get_intrinsics()
 
print(f"焦距: fx={intrinsics.fx}, fy={intrinsics.fy}")
print(f"主点坐标: ppx={intrinsics.ppx}, ppy={intrinsics.ppy}")
print(f"畸变系数: {intrinsics.coeffs}")

焦距: fx=645.2243041992188, fy=644.3118896484375
主点坐标: ppx=633.8898315429688, ppy=360.0555419921875
畸变系数: [-0.05372624471783638, 0.05638597533106804, 0.000343952386174351, 0.0005959899281151593, -0.01772184669971466]


In [12]:
# ======================
# 配置参数
# ======================
SAVE_DIR = "calib_images"
IMG_PREFIX = "img"
MAX_IMAGES = 30

WIDTH = 1280
HEIGHT = 720
FPS = 30

# ======================
# 创建保存目录
# ======================
os.makedirs(SAVE_DIR, exist_ok=True)

# ======================
# RealSense 初始化
# ======================
pipeline = rs.pipeline()
config = rs.config()

# 只启用 RGB
config.enable_stream(
    rs.stream.color,
    WIDTH,
    HEIGHT,
    rs.format.bgr8,
    FPS
)

print("[INFO] Starting RealSense pipeline...")
profile = pipeline.start(config)

# 等待自动曝光稳定
for _ in range(30):
    pipeline.wait_for_frames()

print("[INFO] Press 'Enter' to save image, 'Esc' to quit")

img_count = 0

try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        color_image = np.asanyarray(color_frame.get_data())

        # 显示提示
        display = color_image.copy()
        cv2.putText(
            display,
            f"Saved: {img_count}/{MAX_IMAGES}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.0,
            (0, 255, 0),
            2
        )
        cv2.putText(
            display,
            "Press 'Enter' to save | 'Esc' to quit",
            (20, HEIGHT - 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2
        )

        cv2.imshow("D405 Calibration Capture", display)
        key = cv2.waitKey(1) & 0xFF

        # 保存图像
        if key == 13:
            filename = f"{IMG_PREFIX}_{img_count:03d}.png"
            path = os.path.join(SAVE_DIR, filename)
            cv2.imwrite(path, color_image)
            print(f"[INFO] Saved {path}")
            img_count += 1
            time.sleep(0.3)  # 防止连拍

        # 退出
        elif key == 27:
            break

        if img_count >= MAX_IMAGES:
            print("[INFO] Reached max images.")
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
    print("[INFO] Pipeline stopped.")


[INFO] Starting RealSense pipeline...
[INFO] Press 'Enter' to save image, 'Esc' to quit
[INFO] Saved calib_images\img_000.png
[INFO] Saved calib_images\img_001.png
[INFO] Saved calib_images\img_002.png
[INFO] Saved calib_images\img_003.png
[INFO] Saved calib_images\img_004.png
[INFO] Saved calib_images\img_005.png
[INFO] Saved calib_images\img_006.png
[INFO] Saved calib_images\img_007.png
[INFO] Saved calib_images\img_008.png
[INFO] Saved calib_images\img_009.png
[INFO] Saved calib_images\img_010.png
[INFO] Saved calib_images\img_011.png
[INFO] Saved calib_images\img_012.png
[INFO] Saved calib_images\img_013.png
[INFO] Saved calib_images\img_014.png
[INFO] Saved calib_images\img_015.png
[INFO] Saved calib_images\img_016.png
[INFO] Saved calib_images\img_017.png
[INFO] Saved calib_images\img_018.png
[INFO] Saved calib_images\img_019.png
[INFO] Saved calib_images\img_020.png
[INFO] Saved calib_images\img_021.png
[INFO] Saved calib_images\img_022.png
[INFO] Saved calib_images\img_023.png
